In [9]:
import numpy as np
from PIL import Image
import cv2
import torch

from depth_anything.dpt import DepthAnything
from depth_anything.util.transform import Resize, NormalizeImage, PrepareForNet
from torchvision.transforms import Compose

In [10]:
model = DepthAnything.from_pretrained("LiheYoung/depth_anything_vits14", local_files_only=True)

In [18]:
from dinov2.models.vision_transformer import vit_small, vit_base, vit_large

In [12]:
vits = vit_small(patch_size=14, init_values=1e-5, img_size=518)

In [13]:
for vits_item in vits.state_dict():
    vits_item_inmodel = vits_item.replace('blocks.0.', 'blocks.')
    if vits_item_inmodel in model.pretrained.state_dict() and vits.state_dict()[vits_item].size() == model.pretrained.state_dict()[vits_item_inmodel].size():
        continue
    else:
        print(vits_item, vits.state_dict()[vits_item].size(), model.pretrained.state_dict()[vits_item_inmodel].size())

In [14]:
for model_item in model.pretrained.state_dict():
    model_item_invit = model_item.replace('blocks.', 'blocks.0.')
    if model_item_invit in vits.state_dict() and vits.state_dict()[model_item_invit].size() == model.pretrained.state_dict()[model_item].size():
        continue
    else:
        try:
            print(model_item, vits.state_dict()[model_item_invit].size(), model.pretrained.state_dict()[model_item].size())
        except:
            print(f'{model_item} not in VitS!', model.pretrained.state_dict()[model_item].size())

In [15]:
from collections import OrderedDict
pretrained_vit = OrderedDict()
for model_item in model.pretrained.state_dict():
    model_item_invit = model_item.replace('blocks.', 'blocks.0.')
    pretrained_vit[model_item_invit] = model.pretrained.state_dict()[model_item]
torch.save(pretrained_vit, 'checkpoints/vits.pth')

for i, item in enumerate(pretrained_vit):
    if i > 10:
        break
    print(item, pretrained_vit[item].size())

cls_token torch.Size([1, 1, 384])
pos_embed torch.Size([1, 1370, 384])
mask_token torch.Size([1, 384])
patch_embed.proj.weight torch.Size([384, 3, 14, 14])
patch_embed.proj.bias torch.Size([384])
blocks.0.0.norm1.weight torch.Size([384])
blocks.0.0.norm1.bias torch.Size([384])
blocks.0.0.attn.qkv.weight torch.Size([1152, 384])
blocks.0.0.attn.qkv.bias torch.Size([1152])
blocks.0.0.attn.proj.weight torch.Size([384, 384])
blocks.0.0.attn.proj.bias torch.Size([384])


In [16]:
pthfile = 'checkpoints/vits.pth'
vits.load_state_dict(torch.load(pthfile))

<All keys matched successfully>

In [19]:
vitb = vit_base(patch_size=14, init_values=1e-5, img_size=518)

model = DepthAnything.from_pretrained("checkpoints/depth_anything_vitb14", local_files_only=True)

pretrained_vit = OrderedDict()
for model_item in model.pretrained.state_dict():
    model_item_invit = model_item.replace('blocks.', 'blocks.0.')
    pretrained_vit[model_item_invit] = model.pretrained.state_dict()[model_item]
torch.save(pretrained_vit, 'checkpoints/vitb.pth')

pthfile = 'checkpoints/vitb.pth'
vitb.load_state_dict(torch.load(pthfile))

Loading weights from local directory


<All keys matched successfully>

In [21]:
vitl = vit_large(patch_size=14, init_values=1e-5, img_size=518)

model = DepthAnything.from_pretrained("checkpoints/depth_anything_vitl14", local_files_only=True)

pretrained_vit = OrderedDict()
for model_item in model.pretrained.state_dict():
    model_item_invit = model_item.replace('blocks.', 'blocks.0.')
    pretrained_vit[model_item_invit] = model.pretrained.state_dict()[model_item]
torch.save(pretrained_vit, 'checkpoints/vitl.pth')

pthfile = 'checkpoints/vitl.pth'
vitl.load_state_dict(torch.load(pthfile))

Loading weights from local directory


<All keys matched successfully>